<a href="https://colab.research.google.com/github/rohbot25/Erohan_Repository/blob/main/soilTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install taichi==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00


In [ ]:
import taichi as ti
import numpy as np
import os
import math

max_steps = 100
dt = 0.001
learning_rate = 0.001
gravity = -9.8
soil_damping = .99 # add soil damping / spring damping
damping = 0.01
stiffness = 3000.0
ground_height = 0.1
sim_size = 512
epsilon = 0.00001

soil_particles = 100

real = ti.f32

startingObjectPositions = []

object_radius = 7/sim_size
#n_objects = 4

#for objectIndex in range(n_objects):

#startingObjectPositions.append( [np.random.random(), np.random.random()* 0.9 + 0.1] )

startingObjectPositions.append( [0.1,ground_height+0.3])
startingObjectPositions.append( [0.1,ground_height+0.4])

startingObjectPositions.append( [0.2,ground_height+0.3])
startingObjectPositions.append( [0.2,ground_height+0.4])

n_objects = len(startingObjectPositions)

springs = []

def add_spring(objA,objB,motorized):
  objAIndex = objA
  objAPos = startingObjectPositions[objAIndex]
  objBIndex = objB
  objBPos = startingObjectPositions[objBIndex]
  xDistAB = objAPos[0] - objBPos[0]
  yDistAB = objAPos[1] - objBPos[1]
  distAB = math.sqrt( xDistAB**2 + yDistAB**2 )
  restingLength = distAB
  springs.append( [objAIndex, objBIndex, restingLength,motorized] )

add_spring(0,1,1)
add_spring(1,2,1)
add_spring(2,3,1)
add_spring(3,0,1)
add_spring(0,2,1)
add_spring(1,3,1)

n_springs = len ( springs )

soil = []
mass = []
for i in range(soil_particles):
  soil.append([np.random.uniform(),np.random.uniform()* 0.15 + 0.1001])
  mass.append(int(np.random.random() * 5) + 5)

#soil.append([0.5,0.5])
#soil.append([0.45,0.5])
#mass.append(5)
#mass.append(5)

ti.init( default_fp = real )

vec = lambda: ti.Vector.field(2, dtype = real)

soilPositions = vec()
soilVelocities = vec()
masses = ti.field(ti.i32)

spring_anchor_a = ti.field(ti.i32)
spring_anchor_b = ti.field(ti.i32)
spring_resting_lengths = ti.field(ti.f32)
spring_actuation = ti.field(ti.i32)

positions = vec()
velocities = vec()
springRestoringForces = vec()
springForcesOnObjects = vec()
weightsSH = ti.field(ti.f32)

n_sin_waves = 10
n_hidden_neurons = 32

center = vec()
goal = vec()

hidden = ti.field(ti.f32)
biasH = ti.field(ti.f32)

def n_sensors():
  return n_sin_waves + 4 * n_objects

weightsHM = ti.field(ti.f32)

actuation = ti.field(ti.f32)

# Declare fields for variables to debug
#normals = vec()
#tangents = vec()
#v1ns = ti.field(dtype=real)
#v1ts = ti.field(dtype=real)
#v2ns = ti.field(dtype=real)
#v2ts = ti.field(dtype=real)
#v1n2s = ti.field(dtype=real)
#v2n2s = ti.field(dtype=real)
#vec1ns = vec()
#vec1ts = vec()
#vec2ns = vec()
#vec2ts = vec()
#overlaps = ti.field(dtype=real)

#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(normals)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(tangents)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v1n2s)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(v2n2s)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec1ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ns)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(vec2ts)
#ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(overlaps)

ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(positions)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(positions.grad)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(velocities)
ti.root.dense(ti.i, n_springs).place(spring_anchor_a,spring_anchor_b,spring_resting_lengths,spring_actuation)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_springs).place(springRestoringForces)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(springForcesOnObjects)
ti.root.dense(ti.ij, (n_hidden_neurons, n_sensors())).place(weightsSH)
ti.root.dense(ti.i, max_steps).place(center)
ti.root.place(goal)
ti.root.dense(ti.ij, (max_steps,n_hidden_neurons)).place(hidden)
ti.root.dense(ti.i, n_hidden_neurons).place(biasH)
ti.root.dense(ti.ij, (n_springs, n_hidden_neurons)).place(weightsHM)
ti.root.dense(ti.ij, (max_steps,n_springs)).place(actuation)

loss = ti.field( dtype=ti.f32, shape = ())

ti.root.lazy_grad()

ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(soilPositions)
ti.root.dense(ti.i, max_steps).dense(ti.j, soil_particles).place(soilVelocities)
ti.root.dense(ti.i, soil_particles).place(masses)

@ti.kernel
def Compute_Loss():
  loss[None] -= positions[max_steps-1,0][0]

@ti.kernel
def Calculate_Center(timeStep: ti.i32):
  for _ in range(1):
    c = ti.Vector( [0.0, 0.0])

    for i in range( n_objects):

      c+= positions[timeStep, i]

    center[timeStep] = c / n_objects
# ------------------------------------------

def Draw(frameOffset):
  for timeStep in range(0,max_steps):

    gui = ti.GUI("Robot" , (512,512), background_color = 0xFFFFFF, show_gui=False)

    #draw the floor
    gui.line( begin = ( 0, ground_height) ,
              end = (1,ground_height) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (0,0) ,
              end = (0,1) ,
              color = 0x0,
              radius = 1)
    gui.line( begin = (1,0) ,
              end = (1,1) ,
              color = 0x0,
              radius = 1)
    #draw the soil
    for particle in range(soil_particles):
      x = soilPositions[timeStep,particle][0]
      y = soilPositions[timeStep,particle][1]
      gui.circle ( (x,y), color = 0x0 , radius = masses[particle] )
    #draw objects
    for objectIndex in range(n_objects):

      x = positions[timeStep,objectIndex][0]
      y = positions[timeStep,objectIndex][1]

      gui.circle ( (x,y), color = 0x0 , radius = 7 )

    #draw the springs
    for springIndex in range(n_springs):
      objectAIndex = spring_anchor_a[springIndex]
      objectBIndex = spring_anchor_b[springIndex]
      posA = positions[timeStep,objectAIndex]
      posB = positions[timeStep,objectBIndex]
      motorInside = springs[springIndex][3]
      if motorInside:
        gui.line( begin = posA, end = posB , color = 0x0, radius = 3)
      else:
        gui.line( begin = posA, end = posB , color = 0x0, radius = 1)

    gui.show( 'test' + str(frameOffset + timeStep) + '.png')

# ----------------------------------------
def Initialize():

  for objectIndex in range(soil_particles):

    soilPositions[0,objectIndex] = soil[objectIndex]

    soilVelocities[0,objectIndex] = [0.0,0.0]

    masses[objectIndex] = mass[objectIndex]

  for objectIndex in range(n_objects):

    positions[0,objectIndex] = startingObjectPositions[objectIndex]

    velocities[0,objectIndex] = [0,-0.1]

# ----------------------------------------
def Simulate():

  for timeStep in range(1,max_steps):

    Step_One(timeStep)
#----------------------------------------
@ti.kernel
def Simulate_Soil(timeStep: ti.i32):
   for objA in range(soil_particles):

      oldPosition = soilPositions[timeStep-1, objA]

      newVelocity = soilVelocities[timeStep-1, objA] + \
      dt * gravity * ti.Vector([0,1])

      newPosition = oldPosition + dt * newVelocity
      collision = False
      for objB in range(soil_particles):
        if (objB != objA):
          posA = soilPositions[timeStep-1,objA]
          posB = soilPositions[timeStep-1,objB]
          dist = posA - posB
          length = dist.norm()

          radiusA = masses[objA] / sim_size
          radiusB = masses[objB] / sim_size
          radiiSum = radiusA + radiusB
          if (length < radiiSum):
            collision = True
            if length < epsilon:
              length = epsilon

            normal = dist / length
            tangent = ti.Vector([-normal[1], normal[0]])

            v1n = ti.Vector.dot(newVelocity, normal)
            v1t = ti.Vector.dot(newVelocity, tangent)
            v2n = ti.Vector.dot(soilVelocities[timeStep-1,objB], normal)
            v2t = ti.Vector.dot(soilVelocities[timeStep-1,objB], tangent)

            v1n2 = (v1n * (masses[objA] - masses[objB]) + 2 * masses[objB] * v2n) / (masses[objA] + masses[objB])
            v2n2 = (v2n * (masses[objB] - masses[objA]) + 2 * masses[objA] * v1n) / (masses[objA] + masses[objB])

            vec1n = v1n2 * normal
            vec1t = v1t * tangent
            vec2n = v2n2 * normal
            vec2t = v2t * tangent

            soilVelocities[timeStep,objA] = (vec1n + vec1t) * (1-soil_damping)
            soilVelocities[timeStep,objB] = (vec2n + vec2t) * (1-soil_damping)
            overlap = (radiiSum - length) / 2
            newPosition = soilPositions[timeStep-1,objA] + overlap * normal
            soilPositions[timeStep, objB] = soilPositions[timeStep-1,objB] - overlap * normal

            while length < radiiSum:
              newPosition += overlap * normal
              soilPositions[timeStep, objB] -= overlap * normal
              length += overlap * 2

                      # Store variables for debugging
            #normals[timeStep, objA][0] = normal[0]
            #normals[timeStep, objA][1] = normal[1]
            #tangents[timeStep, objA][0] = tangent[0]
            #tangents[timeStep, objA][1] = tangent[1]
            #v1ns[timeStep, objA] = v1n
            #v1ts[timeStep, objA] = v1t
            #v2ns[timeStep, objA] = v2n
            #v2ts[timeStep, objA] = v2t
            #v1n2s[timeStep, objA] = v1n2
            #v2n2s[timeStep, objA] = v2n2
            #vec1ns[timeStep, objA][0] = vec1n[0]
            #vec1ns[timeStep, objA][1] = vec1n[1]
            #vec1ts[timeStep, objA][0] = vec1t[0]
            #vec1ts[timeStep, objA][1] = vec1t[1]
            #vec2ns[timeStep, objA][0] = vec2n[0]
            #vec2ns[timeStep, objA][1] = vec2n[1]
            #vec2ts[timeStep, objA][0] = vec2t[0]
            #vec2ts[timeStep, objA][1] = vec2t[1]
            #overlaps[timeStep, objA] = overlap

      particle_radius = masses[objA] / sim_size
      if newPosition[0] > 1 - particle_radius:
            newPosition[0] = 1 - particle_radius
            newVelocity[0] *= -1 * (1-soil_damping)

      if newPosition[0] < particle_radius:
            newPosition[0] = particle_radius
            newVelocity[0] *= -1 * (1-soil_damping)

      if newPosition[1] < ground_height + particle_radius:
            newPosition[1] = ground_height + particle_radius
            newVelocity[1] *= -1 * (1-soil_damping)

      if not collision:
        soilVelocities[timeStep,objA] = newVelocity
        soilPositions[timeStep,objA] = newPosition



def Step_One(timeStep: ti.i32):

  Calculate_Center(timeStep)
  Simulate_Neural_Network_SH(timeStep)
  Simulate_Neural_Network_HM(timeStep)
  Simulate_Springs(timeStep)
  Simulate_Objects(timeStep)


# ----------------------------------------

def Make_Movie(frame_rate):
    os.system("rm movie.mp4")
    os.system(f"ffmpeg -r {frame_rate} -i test%d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p movie.mp4")


# ----------------------------------------
def Run_Simulation():
  Initialize()
  for timeStep in range(1, max_steps):
    Simulate_Soil(timeStep)
    #with ti.ad.Tape(loss):
      #Step_One(timeStep)
      #loss[None] = 0
      #Compute_Loss()

  #print(loss[None])


Run_Simulation()
os.system("rm *.png")
Draw(0)


Make_Movie(30)

# Debug printing function
def print_debug_info():
    for timeStep in range(1, max_steps):
        for objA in range(soil_particles):
            print(f"Time Step: {timeStep}, Object: {objA}")
            print(f"Normal: {normals[timeStep, objA]}")
            print(f"Tangent: {tangents[timeStep, objA]}")
            print(f"v1n: {v1ns[timeStep, objA]}")
            print(f"v1t: {v1ts[timeStep, objA]}")
            print(f"v2n: {v2ns[timeStep, objA]}")
            print(f"v2t: {v2ts[timeStep, objA]}")
            print(f"v1n2: {v1n2s[timeStep, objA]}")
            print(f"v2n2: {v2n2s[timeStep, objA]}")
            print(f"vec1n: {vec1ns[timeStep, objA]}")
            print(f"vec1t: {vec1ts[timeStep, objA]}")
            print(f"vec2n: {vec2ns[timeStep, objA]}")
            print(f"vec2t: {vec2ts[timeStep, objA]}")
            print(f"Overlap: {overlaps[timeStep, objA]}")
            print(f"Updated Position: {soilPositions[timeStep, objA]}")
            print(f"Updated Velocity: {soilVelocities[timeStep, objA]}")
            print("------------------")


#print_debug_info()

#watch movie
from IPython.display import HTML
from base64 import b64encode
mp4 = open('movie.mp4', 'rb').read()
data_url = "data:video/mp4;base64,"+ b64encode(mp4).decode()

HTML('<video width=sim_size controls> <source src="%s" type="video/mp4"></video>' % data_url)



[Taichi] Starting on arch=x64
Optimization Step: 0
